# Create EDD Study Files From Data

This notebook creates the files needed for importing a study into Experiment Data Depot (EDD).

## Inputs and outputs

### Required file to run this notebook:
   - `../data/DBTL7/media_descriptions.csv` - media designs for each of the wells
   
   - `../data/DBTL7/OD.xlsx` - production data from the plate reader


### File generated by running this notebook:
   - `edd_experiment_description.csv`
   
   - `edd_protocol.csv`
 
    
The files are stored in the user defined directory.

## Setup

Importing needed libraries:

In [3]:
import sys
sys.path.append('../media_compiler')

import pandas as pd
import openpyxl

from core import create_media_description

### User parameters

In [4]:
CYCLE = 2.1

user_params = {
    'media_file': f'../flaviolin data/DBTL{CYCLE}/media_descriptions.csv',  
    'measurement_file': f'../flaviolin data/DBTL{CYCLE}/OD.xlsx',
    'output_file_path': f'../flaviolin data/DBTL{CYCLE}', # Folder for output files,
    'num_replicates': 3,
    'num_designs': 16,
    'protocol_name': ['OD600', 'OD340'],
    'time_point': 48,
    'part_id': 'JBx_193086',
    'media': 'MOPS',
    'culture_volume': 15,
    'well_volume': 1500,
    'shaking_speed': 800,
    'temperature': 30,
    } 


In [5]:
df = pd.read_csv(user_params['media_file'], index_col=0)
df.head()

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l]
Well,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.036568,20.0,1.388060,1.010494,0.010567,13.877020,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05
A2,40.0,4.0,0.036568,20.0,1.388060,1.010494,0.010567,13.877020,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05
A3,40.0,4.0,0.036568,20.0,1.388060,1.010494,0.010567,13.877020,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05
A4,40.0,4.0,0.004091,20.0,0.285941,1.114373,0.072114,8.571532,0.080340,334.835565,0.000276,0.000065,0.0001,0.0008,0.0001,0.05
A5,40.0,4.0,0.004091,20.0,0.285941,1.114373,0.072114,8.571532,0.080340,334.835565,0.000276,0.000065,0.0001,0.0008,0.0001,0.05


## Create Line Description

In [6]:
df['Line Description'] = df.apply(create_media_description, axis=1)


In [7]:
df['Line Description']

Well
A1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A4    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A5    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A6    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A7    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
A8    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B2    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B3    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B4    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B5    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B6    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B7    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
B8    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C1    MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...
C2    MOPS[mM]: 40.000000, Tricine[mM]: 4.0

## Create Line Names

Add metadata for media and replicates to craft Line Names as **C[.]\_W[.]1\_[.]3-R[.]** denoting cycle number, wells occupying the same design and replicate number:

Check if it's row or column order of lines:

In [8]:
column_order = True if df.index[1][0] == 'B' else False

In [9]:
reps = user_params['num_replicates']
num_media_designs = user_params['num_designs']

def linefunction_colum_order(row):
    well = row.name
    if well[0] in 'ABC':
        return f'C{CYCLE}_WA{well[1]}_C{well[1]}-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_WD{well[1]}_F{well[1]}-R{row["Replicate"]}'                                                                                              

def linefunction_row_order(row):
    well = row.name
    if int(well[1]) < 5:
        return f'C{CYCLE}_W{well[0]}1_{well[0]}4-R{row["Replicate"]}'
    else:
        return f'C{CYCLE}_W{well[0]}5_{well[0]}8-R{row["Replicate"]}'      
    
def linefunction_row_order_triplicates(row):
    well = row.name

    well_names = list(df.index)
    index = well_names.index(well)
    rem = index%3
    name = f'C{CYCLE}_W{well_names[index-rem]}_{well_names[index-rem+2]}-R{row["Replicate"]}'
    print(name)
    return name
    
    

    
df['Replicate'] = [i+1 for _ in range(num_media_designs) for i in range(reps)]
df['Line Name'] = df.apply(linefunction_row_order_triplicates, axis=1)

# if column_order:
#     df['Line Name'] = df.apply(linefunction_colum_order, axis=1)
# else:
#     df['Line Name'] = df.apply(linefunction_row_order, axis=1)

C2.1_WA1_A3-R1
C2.1_WA1_A3-R2
C2.1_WA1_A3-R3
C2.1_WA4_A6-R1
C2.1_WA4_A6-R2
C2.1_WA4_A6-R3
C2.1_WA7_B1-R1
C2.1_WA7_B1-R2
C2.1_WA7_B1-R3
C2.1_WB2_B4-R1
C2.1_WB2_B4-R2
C2.1_WB2_B4-R3
C2.1_WB5_B7-R1
C2.1_WB5_B7-R2
C2.1_WB5_B7-R3
C2.1_WB8_C2-R1
C2.1_WB8_C2-R2
C2.1_WB8_C2-R3
C2.1_WC3_C5-R1
C2.1_WC3_C5-R2
C2.1_WC3_C5-R3
C2.1_WC6_C8-R1
C2.1_WC6_C8-R2
C2.1_WC6_C8-R3
C2.1_WD1_D3-R1
C2.1_WD1_D3-R2
C2.1_WD1_D3-R3
C2.1_WD4_D6-R1
C2.1_WD4_D6-R2
C2.1_WD4_D6-R3
C2.1_WD7_E1-R1
C2.1_WD7_E1-R2
C2.1_WD7_E1-R3
C2.1_WE2_E4-R1
C2.1_WE2_E4-R2
C2.1_WE2_E4-R3
C2.1_WE5_E7-R1
C2.1_WE5_E7-R2
C2.1_WE5_E7-R3
C2.1_WE8_F2-R1
C2.1_WE8_F2-R2
C2.1_WE8_F2-R3
C2.1_WF3_F5-R1
C2.1_WF3_F5-R2
C2.1_WF3_F5-R3
C2.1_WF6_F8-R1
C2.1_WF6_F8-R2
C2.1_WF6_F8-R3


In [10]:
df.head(5)

,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name
Well,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.036568,20.0,1.388060,1.010494,0.010567,13.877020,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2.1_WA1_A3-R1
A2,40.0,4.0,0.036568,20.0,1.388060,1.010494,0.010567,13.877020,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2.1_WA1_A3-R2
A3,40.0,4.0,0.036568,20.0,1.388060,1.010494,0.010567,13.877020,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",3,C2.1_WA1_A3-R3
A4,40.0,4.0,0.004091,20.0,0.285941,1.114373,0.072114,8.571532,0.080340,334.835565,0.000276,0.000065,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2.1_WA4_A6-R1
A5,40.0,4.0,0.004091,20.0,0.285941,1.114373,0.072114,8.571532,0.080340,334.835565,0.000276,0.000065,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2.1_WA4_A6-R2


## Process measurement files

Read measurements file:

In [11]:
df_600 = pd.read_excel(user_params['measurement_file'], sheet_name='600', index_col=0)
df_340 = pd.read_excel(user_params['measurement_file'], sheet_name='340', index_col=0)

# df_600

In [12]:
df_340

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.3148,0.3316,0.3383,0.5674,0.6308,0.6108,0.1121,0.3267,0.1049,0.1027,0.1008,0.0985
B,0.2665,0.4379,0.4087,0.4020,0.6106,0.6128,0.6194,0.1228,0.1017,0.0984,0.0969,0.0935
C,0.6679,0.6875,0.5357,0.4734,0.4912,0.4673,0.4875,0.4858,0.0931,0.0939,0.0907,0.0888
D,0.3369,0.3399,0.3354,0.3026,0.3139,0.3220,0.3172,0.3345,0.0868,0.0875,0.0846,0.0813
E,0.3307,0.3438,0.4572,0.4614,0.3064,0.2847,0.3288,0.2428,0.0822,0.0823,0.0793,0.0755
F,0.2258,0.2276,0.5736,0.5609,0.5645,0.3586,0.0915,0.3843,0.0791,0.0771,0.0758,0.0730
G,0.0727,0.0747,0.0773,0.0788,0.0814,0.0800,0.0794,0.0805,0.0778,0.0769,0.0740,0.0719
H,0.0721,0.0734,0.0760,0.0765,0.0755,0.0755,0.0752,0.0756,0.0769,0.0753,0.0735,0.0719


### Process OD600

Normalize the data to the control well A9 value (water content):

In [13]:
zero_value_600 = df_600.at['A', 9]
df_600.loc[:, df_600.columns] -= zero_value_600
df_600

,1,2,3,4,5,6,7,8,9,10,11,12
A,0.6636,0.7962,0.8069,0.4797,0.4525,0.4267,0.0271,0.3248,0.0000,0.0088,0.0089,0.0083
B,0.1669,0.8316,0.8204,0.8117,0.7020,0.6504,0.6520,0.0067,0.0061,0.0050,0.0056,0.0042
C,0.8747,0.8801,0.8583,0.9413,0.8723,0.2777,0.0443,0.0354,-0.0004,0.0005,-0.0008,-0.0004
D,0.0315,0.0361,0.0411,0.6932,0.6019,0.5145,0.4222,0.6122,-0.0056,-0.0055,-0.0062,-0.0066
E,0.6334,0.5082,0.6933,0.7280,0.4061,0.3309,0.0171,0.0007,-0.0102,-0.0103,-0.0104,-0.0113
F,0.0473,0.1902,0.7822,0.8159,0.8270,0.0100,-0.0103,0.0128,-0.0136,-0.0133,-0.0141,-0.0139
G,-0.0140,-0.0149,-0.0141,-0.0141,-0.0140,-0.0139,-0.0139,-0.0138,-0.0142,-0.0143,-0.0147,-0.0150
H,-0.0119,-0.0130,-0.0126,-0.0133,-0.0135,-0.0136,-0.0136,-0.0137,-0.0134,-0.0137,-0.0137,-0.0141


Set negative values to zero, multiply the values by 10 to account for 10x dilution and keep only 8 columns and 6 rows:

In [14]:
df_600[df_600 < 0] = 0
df_600.loc[:, df_600.columns] *= 10
df_600 = df_600.iloc[:6,:8]
df_600

,1,2,3,4,5,6,7,8
A,6.636,7.962,8.069,4.797,4.525,4.267,0.271,3.248
B,1.669,8.316,8.204,8.117,7.020,6.504,6.520,0.067
C,8.747,8.801,8.583,9.413,8.723,2.777,0.443,0.354
D,0.315,0.361,0.411,6.932,6.019,5.145,4.222,6.122
E,6.334,5.082,6.933,7.280,4.061,3.309,0.171,0.007
F,0.473,1.902,7.822,8.159,8.270,0.100,0.000,0.128


### Process OD340

Do the same for OD340, except of multiplication by 10, as the OD340 measurements were taken from non-diluted samples:

In [15]:
zero_value_340 = df_340.at['A', 9]
df_340.loc[:, df_340.columns] -= zero_value_340
df_340[df_340 < 0] = 0
df_340 = df_340.iloc[:6,:8]
df_340

,1,2,3,4,5,6,7,8
A,0.2099,0.2267,0.2334,0.4625,0.5259,0.5059,0.0072,0.2218
B,0.1616,0.3330,0.3038,0.2971,0.5057,0.5079,0.5145,0.0179
C,0.5630,0.5826,0.4308,0.3685,0.3863,0.3624,0.3826,0.3809
D,0.2320,0.2350,0.2305,0.1977,0.2090,0.2171,0.2123,0.2296
E,0.2258,0.2389,0.3523,0.3565,0.2015,0.1798,0.2239,0.1379
F,0.1209,0.1227,0.4687,0.4560,0.4596,0.2537,0.0000,0.2794


Transform wide to long format, matching the order of wells in index to the one from `df`:

In [16]:
if column_order:
    indfcn = lambda x: f'{x.name}{int(x["variable"])}'
    # df_600 = df_600.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.melt(ignore_index=False, value_name='OD340')
else:
    indfcn = lambda x: f'{x["variable"]}{(x.name)}'
    # df_600 = df_600.T.melt(ignore_index=False, value_name='OD600')
    df_340 = df_340.T.melt(ignore_index=False, value_name='OD340')

# df_600['Well'] = df_600.apply(indfcn, axis=1)
df_340['Well'] = df_340.apply(indfcn, axis=1)

# df_600.index = df_600['Well']
df_340.index = df_340['Well']

# df_600.drop(columns=['variable', 'Well'], inplace=True)
df_340.drop(columns=['variable', 'Well'], inplace=True)

# df_600.head()

Add measurements to the main dataframe:

In [17]:
# measurOD600 = user_params['protocol_name'][0]
measurOD340 = user_params['protocol_name'][1]
# df[measurOD600] = df_600
df[measurOD340] = df_340
df.head(2)


,MOPS[mM],Tricine[mM],H3BO3[mM],Glucose[mM],K2SO4[mM],K2HPO4[mM],FeSO4[mM],NH4Cl[mM],MgCl2[mM],NaCl[mM],(NH4)6Mo7O24[mM],CoCl2[mM],CuSO4[mM],MnSO4[mM],ZnSO4[mM],Kan[g/l],Line Description,Replicate,Line Name,OD340
Well,,,,,,,,,,,,,,,,,,,,
A1,40.0,4.0,0.036568,20.0,1.38806,1.010494,0.010567,13.87702,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",1,C2.1_WA1_A3-R1,0.2099
A2,40.0,4.0,0.036568,20.0,1.38806,1.010494,0.010567,13.87702,0.762363,44.602736,0.000013,0.000185,0.0001,0.0008,0.0001,0.05,"MOPS[mM]: 40.000000, Tricine[mM]: 4.000000, H3...",2,C2.1_WA1_A3-R2,0.2267


## Create EDD Experiment Description File

In [18]:
df['Media'] = user_params['media']
df['Part ID'] = user_params['part_id']
df['Culture Volume'] = user_params['culture_volume']
df['Flask Volume'] = user_params['well_volume']
df['Growth Temperature'] = user_params['temperature']
df['Shaking speed'] = user_params['shaking_speed']
# df['Starting OD'] =
# df['Replicate Count'] = 24


In [19]:
# Invalid columns for now in EDD
# df['Humidity[%]'] = user_params['humidity']
# df['Plate'] = user_params['plate']

In [20]:
exp_descr_file = f'{user_params["output_file_path"]}/edd_experiment_description.xlsx'
df[['Line Name',
    'Line Description',
    'Part ID',
    'Media',
    'Culture Volume',
    'Flask Volume',
    'Growth Temperature',
    'Shaking speed',
]].to_excel(exp_descr_file, index=False)

## Create EDD Measurement File

OD600

In [21]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD600}.xlsx'
# df['Measurement Type'] = 'Optical Density'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD600]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD340

In [22]:
# measurement_file = f'{user_params["output_file_path"]}/edd_{measurOD340}.xlsx'

# df['Time'] = user_params['time_point']
# df['Value'] = df[measurOD340]
# df['Units'] = 'n/a'
# df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)

OD

In [23]:
measurement_file = f'{user_params["output_file_path"]}/edd_OD.xlsx'
df['Measurement Type'] = measurOD340

df['Time'] = user_params['time_point']
df['Value'] = df[measurOD340]
df['Units'] = 'n/a'
df[['Line Name', 'Measurement Type', 'Time', 'Value', 'Units']].to_excel(measurement_file, index=False)